In [5]:
import torch

In [6]:
import torch

def create_graph_coloring_ising_hamiltonian(adj_matrix, k):
    """
    Create an Ising Hamiltonian for the graph coloring problem.
    
    Parameters:
    -----------
    adj_matrix : torch.Tensor
        Adjacency matrix of shape (n, n) where n is the number of nodes.
        adj_matrix[i, j] = 1 if nodes i and j are adjacent, 0 otherwise.
    k : int
        Number of colors.
        
    Returns:
    --------
    torch.Tensor
        Symmetric tensor representing the Ising Hamiltonian of shape (n*k, n*k).
    """
    n = adj_matrix.shape[0]
    
    # Initialize the Hamiltonian matrix
    H = torch.zeros((n*k, n*k), dtype=torch.float)
    
    # Constraint 1: Each node must have exactly one color
    # Add penalty for a node having multiple colors or no color
    for i in range(n):
        for c1 in range(k):
            idx1 = i * k + c1
            
            # Diagonal terms for one-hot constraint
            H[idx1, idx1] = -1
            
            # Cross terms to penalize multiple colors for the same node
            for c2 in range(c1+1, k):
                idx2 = i * k + c2
                H[idx1, idx2] = 2
                H[idx2, idx1] = 2
    
    # Constraint 2: Adjacent nodes cannot have the same color
    # Add penalty for adjacent nodes having the same color
    for i in range(n):
        for j in range(i+1, n):
            if adj_matrix[i, j] > 0:  # If nodes are adjacent
                for c in range(k):
                    idx_i = i * k + c
                    idx_j = j * k + c
                    
                    # Penalty for same color
                    H[idx_i, idx_j] = 2
                    H[idx_j, idx_i] = 2
    
    return H

def decode_solution(x, n, k):
    """
    Decode the solution from the binary vector.
    
    Parameters:
    -----------
    x : torch.Tensor
        Binary vector of shape (n*k,) representing the solution.
    n : int
        Number of nodes.
    k : int
        Number of colors.
        
    Returns:
    --------
    torch.Tensor
        Coloring assignment of shape (n,) where each element is the color index.
    """
    x_reshaped = x.reshape(n, k)
    colors = torch.argmax(x_reshaped, dim=1)
    return colors

def check_solution(adj_matrix, colors):
    """
    Check if the coloring is valid.
    
    Parameters:
    -----------
    adj_matrix : torch.Tensor
        Adjacency matrix of shape (n, n).
    colors : torch.Tensor
        Color assignments of shape (n,).
        
    Returns:
    --------
    bool
        True if the coloring is valid, False otherwise.
    """
    n = adj_matrix.shape[0]
    
    # Check that adjacent nodes have different colors
    for i in range(n):
        for j in range(i+1, n):
            if adj_matrix[i, j] > 0 and colors[i] == colors[j]:
                return False
    
    return True

# Example usage
# def example():
# Create a simple graph with 4 nodes
n = 4
k = 3  # 3 colors

# Create an adjacency matrix for a cycle graph
adj_matrix = torch.zeros((n, n))
adj_matrix[0, 1] = adj_matrix[1, 0] = 1
adj_matrix[1, 2] = adj_matrix[2, 1] = 1
adj_matrix[2, 3] = adj_matrix[3, 2] = 1
adj_matrix[3, 0] = adj_matrix[0, 3] = 1

# Create the Hamiltonian
H = create_graph_coloring_ising_hamiltonian(adj_matrix, k)
print(f"Hamiltonian shape: {H.shape}")

# In a real scenario, you would solve this using quantum annealing 
# or classical optimization techniques

# For demonstration, let's create a valid coloring manually
# (0 -> color 0, 1 -> color 1, 2 -> color 0, 3 -> color 2)
x = torch.zeros(n * k)
x[0] = 1  # Node 0, Color 0
x[k+1] = 1  # Node 1, Color 1
x[2*k] = 1  # Node 2, Color 0
x[3*k+2] = 1  # Node 3, Color 2

# Calculate the energy
energy = x @ H @ x
print(f"Energy for valid coloring: {energy.item()}")

# Decode the solution
colors = decode_solution(x, n, k)
print(f"Coloring: {colors}")

# Check if the solution is valid
is_valid = check_solution(adj_matrix, colors)
print(f"Is valid coloring: {is_valid}")

# Try an invalid coloring (0 -> color 0, 1 -> color 0, 2 -> color 1, 3 -> color 2)
x_invalid = torch.zeros(n * k)
x_invalid[0] = 1  # Node 0, Color 0
x_invalid[k] = 1  # Node 1, Color 0 (same as node 0, which is adjacent)
x_invalid[2*k+1] = 1  # Node 2, Color 1
x_invalid[3*k+2] = 1  # Node 3, Color 2

# Calculate the energy
energy_invalid = x_invalid @ H @ x_invalid
print(f"Energy for invalid coloring: {energy_invalid.item()}")

# Decode the solution
colors_invalid = decode_solution(x_invalid, n, k)
print(f"Invalid coloring: {colors_invalid}")

# Check if the solution is valid
is_valid_invalid = check_solution(adj_matrix, colors_invalid)
print(f"Is valid coloring for invalid solution: {is_valid_invalid}")

Hamiltonian shape: torch.Size([12, 12])
Energy for valid coloring: -4.0
Coloring: tensor([0, 1, 0, 2])
Is valid coloring: True
Energy for invalid coloring: 0.0
Invalid coloring: tensor([0, 0, 1, 2])
Is valid coloring for invalid solution: False


In [10]:
energy

tensor(-4.)

In [8]:
# adj_matrix = torch.tensor([[0, 1, 1, 1, 1, 1, 1, 1],
#          [1, 0, 1, 1, 1, 1, 1, 1],
#          [1, 1, 0, 1, 1, 1, 1, 1],
#          [1, 1, 1, 0, 1, 1, 1, 1],
#          [1, 1, 1, 1, 0, 1, 1, 1],
#          [1, 1, 1, 1, 1, 0, 1, 1],
#          [1, 1, 1, 1, 1, 1, 0, 1],
#          [1, 1, 1, 1, 1, 1, 1, 0]], dtype=torch.float)
# k = 3

# J = create_graph_coloring_ising_hamiltonian(adj_matrix, k)


# n = adj_matrix.shape[0]

# decode_solution(J, n, k)